<a href="https://colab.research.google.com/github/jacmal/CRYPTO-Returns-Strategy-Analysis/blob/main/trading_strategies_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import json
import requests
import io

from collections import deque
from datetime import datetime
from scipy.optimize import brute

from abc import ABC

In [ ]:
# TODO !!!

# Crypto Datasets URLS
# Daily prices datasets
url_btc_d = 'https://raw.githubusercontent.com/jacmal/CRYPTO-Returns-Strategy-Analysis/main/datasets/BTC-USD_daily.csv'
url_eth_d = 'https://raw.githubusercontent.com/jacmal/CRYPTO-Returns-Strategy-Analysis/main/datasets/ETH-USD_daily.csv'
url_bnb_d = 'https://raw.githubusercontent.com/jacmal/CRYPTO-Returns-Strategy-Analysis/main/datasets/BNB-USD_daily.csv'
# Hourly prices datasets
url_btc_h = 'https://raw.githubusercontent.com/jacmal/CRYPTO-Returns-Strategy-Analysis/main/datasets/Binance_BTCUSDT_1h.csv'
url_eth_h = 'https://raw.githubusercontent.com/jacmal/CRYPTO-Returns-Strategy-Analysis/main/datasets/Binance_ETHUSDT_1h.csv'
url_bnb_h = 'https://raw.githubusercontent.com/jacmal/CRYPTO-Returns-Strategy-Analysis/main/datasets/Binance_BNBUSDT_1h.csv'

# DEV load data sets
#url_btc = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_BTCUSDT_1h.csv"
#url_eth = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_ETHUSDT_1h.csv"
##url_bnb = "https://raw.githubusercontent.com/jacmal/crypto/main/data_sets/Binance_BNBUSDT_1h.csv"

site_btc = requests.get(url_btc_d).content
#site_eth = requests.get(url_eth_d).content

df_btc = pd.read_csv(io.StringIO(site_btc.decode('utf-8')))
#df_eth = pd.read_csv(io.StringIO(site_eth.decode('utf-8')))

# reverse data sets
#df_btc = df_btc.loc[::-1].reset_index(drop=True)
#df_btc = df_btc.iloc[1:,:]

#df_eth = df_eth.loc[::-1].reset_index(drop=True)
#df_eth = df_eth.iloc[1:,:]

#
#col_to_drop = ['unix', 'symbol', 'Volume USDT', 'tradecount', 'Volume BTC']

#df_btc.drop(labels=col_to_drop, axis=1, inplace=True)
#df_eth.drop(labels=col_to_drop, axis=1, inplace=True)

#df_btc.set_index('date', drop=True, inplace=True)
#df_eth.set_index('date', drop=True, inplace=True)

#df_btc['Open_btc'] = df_btc['open'].copy()

df_btc.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700


In [ ]:
df = df_btc[['Date', 'Close']].copy()

In [ ]:
class StartegyOptimizer(ABC):
    def __init__(self):
        self.df = None
        self.df_window = None
        self.start = None
        self.end = None
        self.strategy_metrics = {}

    def load_dataframe(self, df, col_name='Close', add_columns=None):
        '''
        Load and prepare DataFrame for selected time window.

        Argumnets:
        ==========
        df (pandas.DataFrame): Financial dataset.
        col_name (str, optional): Price column name. Default to "Close"
        normalize (bool, optional): Normalize prices in "col_name" column. Default to "False"

        Returns: Not return anything.
        ========
        '''
        if add_columns == None:
            data = df[['Date', col_name]].copy()
        else:
            col_names = ['Date', col_name]
            for c_name in add_columns:
                col_names.append(c_name)
            data = df[col_names].copy()
        
        data.set_index('Date', inplace=True)
        if self.start == None:
            data_window = data
        else:
            data_window = data.loc[self.start:self.end, :]
        
        self.df_window = data_window.copy()
        self.df_window.rename(columns={col_name: 'price'}, inplace=True)

In [ ]:
class SMA_StrategyOptimizer(StartegyOptimizer):
    def __init__(self, SMA_short=None, SMA_long=None, date_start=None, date_end=None):
        super().__init__()
        self.SMA_s = SMA_short
        self.SMA_l = SMA_long

# TODO
    def sma_features(self):
        '''
        Create SMA data features and add them into DataFrame time window.

        Argumnets: No argumnets.
        ==========

        Returns: Not return anything. Updates DataFrame time window variable.
        ========
        '''
        df_temp = self.df_window.copy()
        df_temp['returns'] = np.log(df_temp.price / df_temp.price.shift(1))
        df_temp['sma_s'] = df_temp.price.rolling(window=self.SMA_s).mean()
        df_temp['sma_l'] = df_temp.price.rolling(window=self.SMA_l).mean()

        self.df_window = df_temp

    # TODO
    def set_sma_params(self, SMA_short=None, SMA_long=None):
        '''
        Set new parameters for SMA rolling windows.
        
        Argumnets:
        ==========
        SMA_short (int, optional): Short SMA window size. Default to "None"
        SMA_long (int, optional): Short SMA window size. Default to "None"

        Returns: Not return anything. Updates SMA time windows parameters.
        ========
        '''
        if SMA_short is not None:
            self.SMA_s = SMA_short
        if SMA_long is not None:
            self.SMA_l = SMA_long

    # TODO
    def test_sma_strategy(self):
        '''
        Tests the strategy for selected SMA rolling windows.
        
        Argumnets: No argumnets.
        ==========

        Returns:
        ========
        (float): Strategy cumulative sum score.
        (float): Strategy cumulative score minus returns cumulative score.
        '''
        df_temp = self.df_window.copy()
        df_temp.dropna(inplace=True)
        
        df_temp['positions'] = np.where(df_temp.sma_s > df_temp.sma_l, 1, -1)
        df_temp['strategy'] = df_temp.positions.shift(1) * df_temp.returns
        df_temp.dropna(inplace=True)
        
        df_temp['creturns'] = df_temp.returns.cumsum().apply(np.exp)
        df_temp['cstrategy'] = df_temp.strategy.cumsum().apply(np.exp)
        
        self.results_sma = df_temp.copy()

        strategy_performance = df_temp.cstrategy.iloc[-1]
        returns_performance = strategy_performance - df_temp.creturns.iloc[-1]
        
        return round(strategy_performance, 6), round(returns_performance, 6)
    
    # TODO
    def update_sma_params_and_run_strategy(self, sma_params):
        '''
        Update SMA rolling windows values and tests trading strategy.
        
        Argumnets:
        ==========
        sma_params (tuple): Tuple with SMA windows values.

        Returns:
        ========
        (float): Strategy score.
        '''
        self.set_sma_params(int(sma_params[0]), int(sma_params[1]))
        self.sma_features()
        metrics = self.test_sma_strategy()
        
        self.strategy_metrics['short sma window'] = sma_params[0]
        self.strategy_metrics['long sma window'] = sma_params[1]
        self.strategy_metrics['cumulative returns'] = metrics[0] - metrics[1]
        self.strategy_metrics['cumulative strategy'] = metrics[0]
        self.strategy_metrics['strategy performance'] = metrics[1]
        
        return -self.test_sma_strategy()[0]

    # TODO
    def optimize_sma_strategy_params(self, short_sma_range, long_sma_range):
        '''
        Search optimal SMA strategy values for a rolling window.
        Display optimal SMA window sizes.
        
        Argumnets:
        ==========
        short_sma_range (tuple): 
        long_sma_range (tuple):
        
        Returns:
        ========
        (pandas.DataFrame): Ready SMA Strategy DataFrame with optimal sizes of SMA window.
        '''
        sma_params_optimized = brute(self.update_sma_params_and_run_strategy, (short_sma_range, long_sma_range), finish=None)

        self.set_sma_params(int(sma_params_optimized[0]), int(sma_params_optimized[1]))
        self.sma_features()
        metrics = self.test_sma_strategy()
        strategy_score = self.update_sma_params_and_run_strategy((int(sma_params_optimized[0]), int(sma_params_optimized[1])))
         
        best_values = list(self.strategy_metrics.values())
        
        print('THE BEST SMA PARAMETERS: ')
        print(' best short sma window size: ', best_values[0])
        print(' best long sma window size: ', best_values[1])
        print(' cumulative returns: ', best_values[2])
        print(' cumulative strategy: ', best_values[3])
        print(' strategy performance: ', best_values[4])
        print()
        
        return self.results_sma
        #return sma_params_optimized, -self.update_sma_params_and_run_strategy(sma_params_optimized)
    
    # TODO
    def plot_sma_strategy(self):
        '''
        Plot strategy resaults.
        
        Argumnets: No argumnets.
        ==========

        Returns: Not return anything.
        ========
        '''
        plt.style.use('seaborn-darkgrid')
        title = f'STRATEGY: SMA short - {self.SMA_s} SMA long - {self.SMA_l}'
        self.results_sma[['creturns', 'cstrategy']].plot(title=title, figsize=(12,8));

In [ ]:
SHORT_SMA_RANGE = (1, 30)
LONG_SMA_RANGE = (2, 40)

START_DATE = '2020-01-01'
END_DATE = '2022-05-13'

tester_sma = SMA_StrategyOptimizer(date_start=START_DATE, date_end=END_DATE)
#tester_sma = SMA_Strategy_Tester()
tester_sma.load_dataframe(df, normalize=False)
tester_sma.optimize_sma_strategy_params((SHORT_SMA_RANGE[0], SHORT_SMA_RANGE[1], 1), (LONG_SMA_RANGE[0], LONG_SMA_RANGE[1], 1))

THE BEST SMA PARAMETERS: 
 best short sma window size:  1
 best long sma window size:  4
 cumulative returns:  72.46991200000002
 cumulative strategy:  2941.586112
 strategy performance:  2869.1162



,price,returns,sma_s,sma_l,positions,strategy,creturns,cstrategy
Date,,,,,,,,
2014-09-21,398.821014,-0.024968,398.821014,406.740250,-1,0.024968,0.975341,1.025282
2014-09-22,402.152008,0.008317,402.152008,401.168251,1,-0.008317,0.983488,1.016790
2014-09-23,435.790985,0.080333,435.790985,411.417000,1,0.080333,1.065754,1.101841
2014-09-24,423.204987,-0.029306,423.204987,414.992249,1,-0.029306,1.034974,1.070019
2014-09-25,411.574005,-0.027868,411.574005,418.180496,-1,-0.027868,1.006530,1.040612
...,...,...,...,...,...,...,...,...
2022-05-09,30296.953125,-0.117055,30296.953125,33974.773438,-1,0.117055,74.093073,2877.144645
2022-05-10,31022.906250,0.023679,31022.906250,32720.269531,-1,-0.023679,75.868436,2809.817873
2022-05-11,28936.355469,-0.069627,28936.355469,31078.870117,-1,0.069627,70.765647,3012.429002


In [ ]:
class EMA_Strategy_Tester(StartegyOptimizer):
    # TODO
    '''
    Class for vectorized back testing Exponential Moving Average (EMA) trading strategy.

    Attributes:
    ===========
    EMA_short (int): Time window for short EMA.
     EMA_long (int): Time window for loang EMA.
        start (str): Starting date-time.
          end (str): Ending date-time.

    Methods:
    ========
    load_dataframe:
        Load and prepare DataFrame for selected time window.
    sma_features:
        Create EMA data features and add them into DataFrame time window.
    set_sma_params:
        Set new parameters for EMA rolling windows.
    test_sma_strategy:
        Tests the strategy for selected EMA rolling windows.
    update_sma_params_and_run_strategy:
        Update EMA rolling windows values and tests trading strategy.
    optimize_sma_strategy_params:
        Search optimal EMA strategy values for a rolling window.
    plot_sma_strategy:
        Plot strategy resaults.
    '''
    def __init__(self, EMA_short=None, EMA_long=None, date_start=None, date_end=None):
        super().__init__()
        self.EMA_s = EMA_short
        self.EMA_l = EMA_long

    # TODO
    def ema_features(self):
        '''
        Create EMA data features and add them into DataFrame time window.

        Argumnets: No argumnets.
        ==========

        Returns: Not return anything. Updates DataFrame time window variable.
        ========
        '''
        df_temp = self.df_window.copy()
        df_temp['returns'] = np.log(df_temp.price / df_temp.price.shift(1))
        df_temp['ema_s'] = df_temp.price.ewm(span=self.EMA_s, min_periods=self.EMA_s).mean()
        df_temp['ema_l'] = df_temp.price.ewm(span=self.EMA_l, min_periods=self.EMA_l).mean()

        self.df_window = df_temp

    # TODO
    def set_ema_params(self, EMA_short=None, EMA_long=None):
        '''
        Set new parameters for EMA rolling windows.
        
        Argumnets:
        ==========
        EMA_short (int, optional): Short EMA window size. Default to "None"
        EMA_long (int, optional): Short EMA window size. Default to "None"

        Returns: Not return anything. Updates EMA time windows parameters.
        ========
        '''
        if EMA_short is not None:
            self.EMA_s = EMA_short
        if EMA_long is not None:
            self.EMA_l = EMA_long

    # TODO
    def test_ema_strategy(self):
        '''
        Tests the strategy for selected EMA rolling windows.
        
        Argumnets: No argumnets.
        ==========

        Returns:
        ========
        strategy_performance (float): Strategy cumulative sum score.
        returns_performance (float): Strategy cumulative score minus returns cumulative score.
        '''
        df_temp = self.df_window.copy()
        df_temp.dropna(inplace=True)
        
        df_temp['positions'] = np.where(df_temp.ema_s > df_temp.ema_l, 1, -1)
        df_temp['strategy'] = df_temp.positions.shift(1) * df_temp.returns
        df_temp.dropna(inplace=True)
        
        df_temp['creturns'] = df_temp.returns.cumsum().apply(np.exp)
        df_temp['cstrategy'] = df_temp.strategy.cumsum().apply(np.exp)
        
        self.results_ema = df_temp.copy()

        strategy_performance = df_temp.cstrategy.iloc[-1]
        returns_performance = strategy_performance - df_temp.creturns.iloc[-1]
        
        return round(strategy_performance, 6), round(returns_performance, 6)
    
    # TODO
    def update_ema_params_and_run_strategy(self, ema_params):
        '''
        Update EMA rolling windows values and tests trading strategy.
        
        Argumnets:
        ==========
        ema_params (tuple): Tuple with EMA windows values.

        Returns:
        ========
        (float): Strategy score.
        '''
        self.set_ema_params(int(ema_params[0]), int(ema_params[1]))
        self.ema_features()
        metrics = self.test_ema_strategy()
        
        self.strategy_metrics['short ema window'] = ema_params[0]
        self.strategy_metrics['long ema window'] = ema_params[1]
        self.strategy_metrics['cumulative returns'] = metrics[0] - metrics[1]
        self.strategy_metrics['cumulative strategy'] = metrics[0]
        self.strategy_metrics['strategy performance'] = metrics[1]
        
        return -self.test_ema_strategy()[0]

    # TODO
    def optimize_ema_strategy_params(self, short_ema_range, long_ema_range):
        '''
        Search optimal EMA strategy values for a rolling window.
        Display optimal EMA window sizes.
        
        Argumnets:
        ==========
        short_ema_range (tuple): 
        long_ema_range (tuple):
        
        Returns:
        ========
        (pandas.DataFrame): Ready EMA Strategy DataFrame with optimal sizes of EMA window.
        '''
        ema_params_optimized = brute(self.update_ema_params_and_run_strategy, (short_ema_range, long_ema_range), finish=None)

        self.set_ema_params(int(ema_params_optimized[0]), int(ema_params_optimized[1]))
        self.ema_features()
        metrics = self.test_ema_strategy()
        strategy_score = self.update_ema_params_and_run_strategy((int(ema_params_optimized[0]), int(ema_params_optimized[1])))
         
        best_values = list(self.strategy_metrics.values())
        
        print('THE BEST EMA PARAMETERS: ')
        print(' best short ema window size: ', best_values[0])
        print(' best long ema window size: ', best_values[1])
        print(' cumulative returns: ', best_values[2])
        print(' cumulative strategy: ', best_values[3])
        print(' strategy performance: ', best_values[4])
        print()
        
        return self.results_ema
        #return sma_params_optimized, -self.update_sma_params_and_run_strategy(sma_params_optimized)
    
    # TODO
    def plot_ema_strategy(self):
        '''
        Plot strategy resaults.
        
        Argumnets: No argumnets.
        ==========

        Returns: Not return anything.
        ========
        '''
        plt.style.use('seaborn-darkgrid')
        title = f'STRATEGY: EMA short - {self.EMA_s} EMA long - {self.EMA_l}'
        self.results_ema[['creturns', 'cstrategy']].plot(title=title, figsize=(12,8));

In [ ]:
class SMAEMA_Strategy_Tester(StartegyOptimizer):
    # TODO
    '''
    Class for vectorized back testing Simple Moving Average (SMA) and Exponential Moving Average (EMA) trading strategy.

    Attributes:
    ===========
    SMA (int): Time window size for SMA.
    EMA (int): Time window size for EMA.
    start (str): Starting date-time.
    end (str): Ending date-time.

    Methods:
    ========
    load_dataframe:
        Load and prepare DataFrame for selected time window.
    sma_features:
        Create SMA nad EMA data features and add them into DataFrame time window.
    set_sma_params:
        Set new parameters for SMA and EMA rolling windows.
    test_sma_strategy:
        Tests the strategy for selected SMA, EMA rolling windows.
    update_sma_params_and_run_strategy:
        Update SMA and EMA rolling windows values and tests trading strategy.
    optimize_sma_strategy_params:
        Search optimal SMAEMA strategy values for a rolling window.
    plot_sma_strategy:
        Plot strategy resaults.
    '''
    def __init__(self, SMA=None, EMA=None, date_start=None, date_end=None):
        super().__init__()
        self.SMA = SMA
        self.EMA = EMA

    # TODO
    def smaema_features(self):
        '''
        Create SMA and EMA data features and add them into DataFrame time window.

        Argumnets: No argumnets.
        ==========

        Returns: Not return anything. Updates DataFrame time window variable.
        ========
        '''
        df_temp = self.df_window.copy()
        df_temp['returns'] = np.log(df_temp.price / df_temp.price.shift(1))
        df_temp['sma'] = df_temp.price.rolling(window=self.SMA).mean()
        df_temp['ema'] = df_temp.price.ewm(span=self.EMA, min_periods=self.EMA).mean()

        self.df_window = df_temp

    # TODO
    def set_smaema_params(self, SMA=None, EMA=None):
        '''
        Set new parameters for SMA and EMA rolling windows.
        
        Argumnets:
        ==========
        SMA (int, optional): SMA window size. Default to "None"
        EMA (int, optional): EMA window size. Default to "None"

        Returns: Not return anything. Updates SMA, EMA time windows parameters.
        ========
        '''
        if SMA is not None:
            self.SMA = SMA
        if EMA is not None:
            self.EMA = EMA

    # TODO
    def test_smaema_strategy(self):
        '''
        Tests the strategy for selected SMAEMA rolling windows.
        
        Argumnets: No argumnets.
        ==========

        Returns:
        ========
        (float): Strategy cumulative sum score.
        (float): Strategy cumulative score minus returns cumulative score.
        '''
        df_temp = self.df_window.copy()
        df_temp.dropna(inplace=True)
        
        df_temp['positions'] = np.where(df_temp.ema > df_temp.sma, 1, -1)
        df_temp['strategy'] = df_temp.positions.shift(1) * df_temp.returns
        df_temp.dropna(inplace=True)
        
        df_temp['creturns'] = df_temp.returns.cumsum().apply(np.exp)
        df_temp['cstrategy'] = df_temp.strategy.cumsum().apply(np.exp)
        
        self.results_smaema = df_temp.copy()

        strategy_performance = df_temp.cstrategy.iloc[-1]
        returns_performance = strategy_performance - df_temp.creturns.iloc[-1]
        
        return round(strategy_performance, 6), round(returns_performance, 6)
    
    # TODO
    def update_smaema_params_and_run_strategy(self, smaema_params):
        '''
        Update SMA and EMA rolling windows values and tests trading strategy.
        
        Argumnets:
        ==========
        smaema_params (tuple): Tuple with SMAEMA window values.

        Returns:
        ========
        (float): Strategy score.
        '''
        self.set_smaema_params(int(smaema_params[0]), int(smaema_params[1]))
        self.smaema_features()
        metrics = self.test_smaema_strategy()
        
        self.strategy_metrics['sma window'] = smaema_params[0]
        self.strategy_metrics['ema window'] = smaema_params[1]
        self.strategy_metrics['cumulative returns'] = metrics[0] - metrics[1]
        self.strategy_metrics['cumulative strategy'] = metrics[0]
        self.strategy_metrics['strategy performance'] = metrics[1]
        
        return -self.test_smaema_strategy()[0]

    # TODO
    def optimize_smaema_strategy_params(self, sma_range, ema_range):
        '''
        Search optimal SMA and EMA strategy values for a rolling window.
        Display optimal SMA, EMA window sizes.
        
        Argumnets:
        ==========
        sma_range (tuple): 
        ema_range (tuple):
        
        Returns:
        ========
        (pandas.DataFrame): Ready SMAEMA Strategy DataFrame with optimal sizes of SMA and EMA windows.
        '''
        smaema_params_optimized = brute(self.update_smaema_params_and_run_strategy, (sma_range, ema_range), finish=None)

        self.set_smaema_params(int(smaema_params_optimized[0]), int(smaema_params_optimized[1]))
        self.smaema_features()
        metrics = self.test_smaema_strategy()
        strategy_score = self.update_smaema_params_and_run_strategy((int(smaema_params_optimized[0]), int(smaema_params_optimized[1])))
         
        best_values = list(self.strategy_metrics.values())
        
        print('THE BEST SMA, EMA PARAMETERS: ')
        print(' best sma window size: ', best_values[0])
        print(' best ema window size: ', best_values[1])
        print(' cumulative returns: ', best_values[2])
        print(' cumulative strategy: ', best_values[3])
        print(' strategy performance: ', best_values[4])
        print()
        
        return self.results_smaema
        #return sma_params_optimized, -self.update_sma_params_and_run_strategy(sma_params_optimized)
    
    # TODO
    def plot_smaema_strategy(self):
        '''
        Plot strategy resaults.
        
        Argumnets: No argumnets.
        ==========

        Returns: Not return anything.
        ========
        '''
        plt.style.use('seaborn-darkgrid')
        title = f'STRATEGY: SMA - {self.SMA} EMA - {self.EMA}'
        self.results_smaema[['creturns', 'cstrategy']].plot(title=title, figsize=(12,8));

In [ ]:
class MACD_Strategy_Tester(StartegyOptimizer):
    # TODO
    '''
    Class for vectorized back testing Moving Average Convergence Divergence (MACD) trading strategy.

    Attributes:
    ===========
    EMA_short (int): Time window for short EMA.
     EMA_long (int): Time window for loang EMA.
        start (str): Starting date-time.
          end (str): Ending date-time.

    Methods:
    ========
    load_dataframe:
        Load and prepare DataFrame for selected time window.
    macd_features:
        Create MACD data features and add them into DataFrame time window.
    set_macd_params:
        Set new parameters for EMA and MACD SIGNAL rolling windows.
    test_macd_strategy:
        Tests the strategy for selected MACD rolling windows.
    update_sma_params_and_run_strategy:
        Update EMA, SIAGNAL MACD rolling windows values and tests trading strategy.
    optimize_sma_strategy_params:
        Search optimal MACD strategy values for a rolling window.
    plot_sma_strategy:
        Plot strategy resaults.
    '''
    def __init__(self, EMA_short=None, EMA_long=None, signal_macd_window=None, date_start=None, date_end=None):
        super().__init__()
        self.EMA_s = EMA_short
        self.EMA_l = EMA_long
        self.signal_macd = signal_macd_window

    # TODO
    def macd_features(self):
        '''
        Create MACD data features and add them into DataFrame time window.

        Argumnets: No argumnets.
        ==========

        Returns: Not return anything. Updates DataFrame time window variable.
        ========
        '''
        df_temp = self.df_window.copy()
        df_temp['returns'] = np.log(df_temp.price / df_temp.price.shift(1))
        df_temp['ema_s'] = df_temp.price.ewm(span=self.EMA_s, min_periods=self.EMA_s).mean()
        df_temp['ema_l'] = df_temp.price.ewm(span=self.EMA_l, min_periods=self.EMA_l).mean()
        df_temp['macd'] = df_temp.ema_s - df_temp.ema_l
        df_temp['signal_macd'] = df_temp.macd.ewm(span=self.signal_macd, min_periods=self.signal_macd).mean()

        self.df_window = df_temp.copy()

    # TODO
    def set_macd_params(self, EMA_short=None, EMA_long=None, signal_macd=None):
        '''
        Set new parameters for EMA rolling windows.
        
        Argumnets:
        ==========
        EMA_short (int, optional): Short EMA window size. Default to "None"
        EMA_long (int, optional): Short EMA window size. Default to "None"

        Returns: Not return anything. Updates EMA time windows parameters.
        ========
        '''
        if EMA_short is not None:
            self.EMA_s = EMA_short
        if EMA_long is not None:
            self.EMA_l = EMA_long
        if signal_macd is not None:
            self.signal_macd = signal_macd

    # TODO
    def test_macd_strategy(self):
        '''
        Tests the strategy for selected EMA rolling windows.
        
        Argumnets: No argumnets.
        ==========

        Returns:
        ========
        strategy_performance (float): Strategy cumulative sum score.
        returns_performance (float): Strategy cumulative score minus returns cumulative score.
        '''
        df_temp = self.df_window.copy()
        df_temp.dropna(inplace=True)
        
        df_temp['positions'] = np.where(df_temp.macd - df_temp.signal_macd > 0, 1, -1)
        df_temp['strategy'] = df_temp.positions.shift(1) * df_temp.returns
        df_temp.dropna(inplace=True)
        
        df_temp['creturns'] = df_temp.returns.cumsum().apply(np.exp)
        df_temp['cstrategy'] = df_temp.strategy.cumsum().apply(np.exp)
        
        self.results_ema = df_temp.copy()

        strategy_performance = df_temp.cstrategy.iloc[-1]
        returns_performance = strategy_performance - df_temp.creturns.iloc[-1]
        
        return round(strategy_performance, 6), round(returns_performance, 6)
    
    # TODO
    def update_macd_params_and_run_strategy(self, macd_params):
        '''
        Update EMA and SIGNAL MACD rolling windows values and tests trading strategy.
        
        Argumnets:
        ==========
        ema_params (tuple): Tuple with EMA windows values.

        Returns:
        ========
        (float): Strategy score.
        '''
        self.set_macd_params(int(macd_params[0]), int(macd_params[1]), int(macd_params[2]))
        self.macd_features()
        metrics = self.test_macd_strategy()
        
        self.strategy_metrics['short ema window'] = macd_params[0]
        self.strategy_metrics['long ema window'] = macd_params[1]
        self.startegy_metrics['signal macd window'] = macd_params[2]
        
        self.strategy_metrics['cumulative returns'] = metrics[0] - metrics[1]
        self.strategy_metrics['cumulative strategy'] = metrics[0]
        self.strategy_metrics['strategy performance'] = metrics[1]
        
        return -self.test_macd_strategy()[0]

    # TODO
    def optimize_macd_strategy_params(self, short_ema_range, long_ema_range, signal_macd_range):
        '''
        Search optimal EMA strategy values for a rolling window.
        Display optimal EMA window sizes.
        
        Argumnets:
        ==========
        short_ema_range (tuple): 
        long_ema_range (tuple):
        
        Returns:
        ========
        (pandas.DataFrame): Ready EMA Strategy DataFrame with optimal sizes of EMA window.
        '''
        macd_params_optimized = brute(self.update_macd_params_and_run_strategy, (short_ema_range, long_ema_range, signal_macd_range), finish=None)

        self.set_macd_params(int(macd_params_optimized[0]), int(macd_params_optimized[1]), int(macd_params_optimized[2]))
        self.macd_features()
        metrics = self.test_macd_strategy()
        strategy_score = self.update_macd_params_and_run_strategy((int(ema_params_optimized[0]), int(ema_params_optimized[1]), int(macd_params_optimized[2])))
         
        best_values = list(self.strategy_metrics.values())
        
        print('THE BEST EMA PARAMETERS: ')
        print(' best short ema window size: ', best_values[0])
        print(' best long ema window size: ', best_values[1])
        print(' best signal macd window size: ', best_values[2])
        print(' cumulative returns: ', best_values[2])
        print(' cumulative strategy: ', best_values[3])
        print(' strategy performance: ', best_values[4])
        print()
        
        return self.results_macd
        #return sma_params_optimized, -self.update_sma_params_and_run_strategy(sma_params_optimized)
    
    # TODO
    def plot_macd_strategy(self):
        '''
        Plot strategy resaults.
        
        Argumnets: No argumnets.
        ==========

        Returns: Not return anything.
        ========
        '''
        plt.style.use('seaborn-darkgrid')
        title = f'STRATEGY: EMA short - {self.EMA_s} EMA long - {self.EMA_l} signal macd - {self.signal_macd}'
        self.results_macd[['creturns', 'cstrategy']].plot(title=title, figsize=(12,8));

In [ ]:
class RSI_StrategyOptimizer(StartegyOptimizer):
    def __init__(self, ma_window=None, rsi_upper=None, rsi_upper=None, date_start=None, date_end=None):
        super().__init__()
        self.ma_window = ma_window
        self.rsi_upper = rsi_upper
        self.rsi_lower = rsi_lower

# TODO
    def rsi_features(self):
        '''
        Create SMA data features and add them into DataFrame time window.

        Argumnets: No argumnets.
        ==========

        Returns: Not return anything. Updates DataFrame time window variable.
        ========
        '''
        df_temp = self.df_window.copy()
        df_temp['returns'] = np.log(df_temp.price / df_temp.price.shift(1))
        df_temp['U'] = np.where(df_temp.price.diff() > 0, df_temp.price.diff(), 0)
        df_temp['D'] = np.where(df_temp.price.diff() < 0, -df_temp.price.diff(), 0)
        df_temp['ma_U'] = df_temp.U.rolling(window=self.ma_window).mean()
        df_temp['ma_D'] = df_temp.D.rolling(window=self.ma_window).mean()
        df_temp['RSI'] = df_temp.ma_U / (df_temp.ma_U + df_temp.ma_D) * 100
        #df_temp['sma_s'] = df_temp.price.rolling(window=self.SMA_s).mean()
        #df_temp['sma_l'] = df_temp.price.rolling(window=self.SMA_l).mean()

        self.df_window = df_temp.copy()

    # TODO
    def set_rsi_params(self, ma_window=None, rsi_upper=None, rsi_lower=None):
        '''
        Set new parameters for SMA rolling windows.
        
        Argumnets:
        ==========
        SMA_short (int, optional): Short SMA window size. Default to "None"
        SMA_long (int, optional): Short SMA window size. Default to "None"

        Returns: Not return anything. Updates SMA time windows parameters.
        ========
        '''
        if ma_window is not None:
            self.ma_window = ma_window
        if rsi_upper is not None:
            self.rsi_upper = rsi_upper
        if rsi_lower is not None:
            self.rsi_lower = rsi_lower

    # TODO
    def test_rsi_strategy(self):
        '''
        Tests the strategy for selected SMA rolling windows.
        
        Argumnets: No argumnets.
        ==========

        Returns:
        ========
        (float): Strategy cumulative sum score.
        (float): Strategy cumulative score minus returns cumulative score.
        '''
        df_temp = self.df_window.copy()
        df_temp.dropna(inplace=True)
        
        df_temp['positions'] = np.where(df_temp.RSI > self.rsi_upper, -1, 0)
        df_temp['positions'] = np.where(df_temp.RSI < self.rsi_lower, 1, df_temp.positions)
        
        df_temp['strategy'] = df_temp.positions.shift(1) * df_temp.returns
        df_temp.dropna(inplace=True)
        
        df_temp['creturns'] = df_temp.returns.cumsum().apply(np.exp)
        df_temp['cstrategy'] = df_temp.strategy.cumsum().apply(np.exp)
        
        self.results_sma = df_temp.copy()

        strategy_performance = df_temp.cstrategy.iloc[-1]
        returns_performance = strategy_performance - df_temp.creturns.iloc[-1]
        
        return round(strategy_performance, 6), round(returns_performance, 6)
    
    # TODO
    def update_rsi_params_and_run_strategy(self, rsi_params):
        '''
        Update SMA rolling windows values and tests trading strategy.
        
        Argumnets:
        ==========
        sma_params (tuple): Tuple with SMA windows values.

        Returns:
        ========
        (float): Strategy score.
        '''
        self.set_rsi_params(int(rsi_params[0]), int(rsi_params[1]), int(rsi_params[2]))
        self.rsi_features()
        metrics = self.test_rsi_strategy()
        
        self.strategy_metrics['mowing average window'] = rsi_params[0]
        self.strategy_metrics['RSI upper'] = rsi_params[1]
        self.strategy_metrics['RSI lower'] = rsi_params[2]

        self.strategy_metrics['cumulative returns'] = metrics[0] - metrics[1]
        self.strategy_metrics['cumulative strategy'] = metrics[0]
        self.strategy_metrics['strategy performance'] = metrics[1]
        
        return -self.test_rsi_strategy()[0]

    # TODO
    def optimize_rsi_strategy_params(self, ma_window_range, rsi_upper_range, rsi_lower_range):
        '''
        Search optimal SMA strategy values for a rolling window.
        Display optimal SMA window sizes.
        
        Argumnets:
        ==========
        short_sma_range (tuple): 
        long_sma_range (tuple):
        
        Returns:
        ========
        (pandas.DataFrame): Ready SMA Strategy DataFrame with optimal sizes of SMA window.
        '''
        rsi_params_optimized = brute(self.update_rsi_params_and_run_strategy, (ma_window_range, rsi_upper_range, rsi_lower_range), finish=None)

        self.set_rsi_params(int(rsi_params_optimized[0]), int(rsi_params_optimized[1]), int(rsi_params_optimized[2]))
        self.rsi_features()
        metrics = self.test_rsi_strategy()
        strategy_score = self.update_rsi_params_and_run_strategy((int(rsi_params_optimized[0]), int(rsi_params_optimized[1]), int(rsi_params_optimized[2])))
         
        best_values = list(self.strategy_metrics.values())
        
        print('THE BEST SMA PARAMETERS: ')
        print(' moving average window size: ', best_values[0])
        print(' RSI upper: ', best_values[1])
        print(' RSI lower: ', best_values[2])
        
        print(' cumulative returns: ', best_values[2])
        print(' cumulative strategy: ', best_values[3])
        print(' strategy performance: ', best_values[4])
        print()
        
        return self.results_rsi
        #return sma_params_optimized, -self.update_sma_params_and_run_strategy(sma_params_optimized)
    
    # TODO
    def plot_sma_strategy(self):
        '''
        Plot strategy resaults.
        
        Argumnets: No argumnets.
        ==========

        Returns: Not return anything.
        ========
        '''
        plt.style.use('seaborn-darkgrid')
        title = f'STRATEGY: moving average window - {self.ma_window} RSI upper - {self.rsi_upper} RSI lower - {self.rsi_lower}'
        self.results_sma[['creturns', 'cstrategy']].plot(title=title, figsize=(12,8));